# Project Overview

## Part 1. Pre-Processing the Files

In [19]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

### Creating list of filepaths to process original event csv data files

In [20]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [21]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [22]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Part 2. Create Keyspaces in Apache Cassandra and import data

### Create Cluster

In [31]:
host = os.environ["HOSTADDR"]

from cassandra.cluster import Cluster
try: 
    cluster = Cluster([host]) 
    session = cluster.connect()
except Exception as e:
    print(e)

### Create Keyspace

In [32]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

### Set Keyspace

In [33]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Query 1:  

Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [34]:
query = (
"""
CREATE TABLE IF NOT EXISTS user_activity \
(sessionId int, itemInSession int, artist text, song_title text, song_length float, PRIMARY KEY (sessionId, itemInSession))
""")

try:
    session.execute(query)
except Exception as e:
    print(e)

In [35]:
file = 'event_datafile_new.csv'

query = ("""
INSERT INTO user_activity (sessionId, itemInSession, artist, song_title, song_length) \
VALUES (%s, %s, %s, %s, %s)
""")

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist_name, song, float(length)))

In [37]:
#SELECT statement to verify the data was entered into the table
query = ("""
SELECT artist, song_title, song_length FROM user_activity WHERE sessionId = 338 AND itemInSession = 4
""")

rows = session.execute(query)

for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Drop the tables before closing out the sessions

In [39]:
query = "drop table user_activity"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection

In [40]:
session.shutdown()
cluster.shutdown()

In [43]:
#Some random debug queries
query = "SELECT * FROM system_schema.keyspaces"
#query = "SELECT COUNT(*) FROM user_activity"
rows = session.execute(query)

for row in rows:
    print(row)

NoHostAvailable: ('Unable to complete the operation against any hosts', {<Host: 52.151.117.78:9042 datacenter1>: ConnectionException('Pool is shutdown')})